In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import numpy as np
import pandas as pd
font = {'size'   : 10}
import matplotlib
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt;
import shutil
import psutil

import sys
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd


In [ ]:
# specify the temporary working folder
t_d = os.path.join('pst_template_opt')
if os.path.exists(t_d):
    shutil.rmtree(t_d)

org_t_d = os.path.join("master_ies0")
if not os.path.exists(org_t_d):
    raise Exception()

shutil.copytree(org_t_d,t_d)

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"pest.pst"))
pe = pst.ies.get("paren",pst.ies.phiactual.iteration.max())

In [ ]:
par = pst.parameter_data
par.loc[pe.columns,"parval1"] = pe.loc["base",:].values.flatten()

In [ ]:
pst.control_data.noptmax = 0
pst.write(os.path.join(t_d,"pest.pst"),version=2)
pyemu.os_utils.run("pestpp-ies pest.pst",cwd=t_d)

In [ ]:
pst.set_res(os.path.join(t_d,"pest.base.rei"))

In [ ]:
forecasts = pst.pestpp_options["forecasts"].split(",")
forecasts.sort()
forecasts

In [ ]:
res = pst.res
res.loc[forecasts,:]

In [ ]:
res.loc[res.name.str.contains("gde"),:]

So our calibrated model is over pumping: the simulated pit level is lower than 80 and the end of mining GDE flux is not even close to the pre-development estimate...

Let's see if we can optimize our way out of this issue

In [ ]:
wpar = par.loc[par.parnme.str.contains("wel"),:]

In [ ]:
wpar.shape

In [ ]:
par.loc[wpar.parnme,"partrans"] = "none"
pst.pestpp_options["opt_dec_var_groups"] = wpar.pargp.unique().tolist()
pargp = pst.parameter_groups
pargp.loc[pst.pestpp_options["opt_dec_var_groups"],"inctyp"] = "absolute"
pargp.loc[pst.pestpp_options["opt_dec_var_groups"],"derinc"] = 250


In [ ]:
obs = pst.observation_data
obs.loc[forecasts[0],"obgnme"] = "less_than"
obs.loc[forecasts[0],"obsval"] = obs.loc[forecasts[0].replace("time:40151","time:1"),"obsval"]
obs.loc[forecasts[0],"weight"] = 1.0

In [ ]:
obs = pst.observation_data
dobs = obs.loc[(obs.obsnme.str.contains("gde")) & (obs.obsnme.str.contains("time:3651")),:]
assert dobs.shape[0] == 1
obs.loc[dobs.obsnme,"weight"] = 1.0
obs.loc[dobs.obsnme,"obgnme"] = "less_than"
obs.loc[dobs.obsnme,"obsval"] = obs.loc[forecasts[0].replace("time:40151","time:1"),"obsval"]


In [ ]:
obs.loc[forecasts[1],"obsval"] = 80
obs.loc[forecasts[1],"obgnme"] = "less_than"
obs.loc[forecasts[1],"weight"] = 1.0

Make sure we arent reinjecting more water than we are extracting

In [ ]:
diffobs = obs.loc[(obs.oname=="inc") & (obs.usecol=="totwel") & (obs.totim=="3651"),:]
assert diffobs.shape[0] == 1
obs.loc[diffobs.obsnme,"weight"] = 1.0
obs.loc[diffobs.obsnme,"obsval"] = 0.0
obs.loc[diffobs.obsnme,"obgnme"] = "less_than"

For an objective function, we want to minimize the total flux of water being used for both dewatering and reinjection.  That value comes from the modflow list file budget and the absolute sum of both wel packages (we added that bit way back in the pstfrom notebook)

In [ ]:
aobs = obs.loc[(obs.oname=="inc") & (obs.usecol=="abstotwel") & (obs.totim=="3651"),:]
assert aobs.shape[0] == 1
objname = aobs.obsnme.values[0]
#obs.loc[objname,"weight"] = 1.0
obs.loc[objname,"obgnme"] = "less_than"
pst.pestpp_options["opt_obj_func"] = objname

In [ ]:
pst.control_data.noptmax = 3

In [ ]:
pst.write(os.path.join(t_d,"pest.pst"),version=2)

In [ ]:
num_workers=30
m_d = "master_opt0"

In [ ]:
pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-opt', #the PEST software version we want to run
                            'pest.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

Ruh roh - our problem is infeasible!  that means its not possible to make the model meet both the strict pit gw level requirement and the post-closure and end-of-mining GDE flux requirement #sad



Let's twist this problem a bit more to extract more valuable information from the analysis.  Let's turn the GDE flux at the end of mining into the objective function to see how high we can make that value.  

In [ ]:
alt_t_d = "pst_template_optalt"
if os.path.exists(alt_t_d):
    shutil.rmtree(alt_t_d)
shutil.copytree(t_d,alt_t_d)
pst = pyemu.Pst(os.path.join(alt_t_d,"pest.pst"))
obs = pst.observation_data

In [ ]:
obs.loc[dobs.obsnme,"weight"] = 0.0
obs.loc[forecasts[0],"weight"] = 0.0
pst.pestpp_options["opt_obj_func"] = dobs.obsnme.values[0]
pst.control_data.noptmax = 3
pst.write(os.path.join(alt_t_d,"pest.pst"),version=2)
m_d = "master_optalt"
pyemu.os_utils.start_workers(alt_t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-opt', #the PEST software version we want to run
                            'pest.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )


Another option is to loosen things up a lil - what if we only need to be within two times 20% of the pre-development GDE flux - it is after all an imprecise measured value and we said the standard deviation was 20%, so two sigma is approximately 95% confidence...

In [ ]:
alt2_t_d = "pst_template_optalt2"
if os.path.exists(alt2_t_d):
    shutil.rmtree(alt2_t_d)
shutil.copytree(t_d,alt2_t_d)
pst = pyemu.Pst(os.path.join(alt2_t_d,"pest.pst"))
obs = pst.observation_data
predev_gde_val = obs.loc[forecasts[0],"obsval"] = obs.loc[forecasts[0].replace("time:40151","time:1"),"obsval"]
print(predev_gde_val)
rhs_val = predev_gde_val - ((predev_gde_val * 0.2)*2)
print(rhs_val)
obs.loc[forecasts[0],"obsval"] = rhs_val
obs.loc[dobs.obsnme,"obsval"] = rhs_val


In [ ]:
pst.write(os.path.join(alt2_t_d,"pest.pst"),version=2)
m_d = "master_optalt2

In [ ]:
pyemu.os_utils.start_workers(alt2_t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-opt', #the PEST software version we want to run
                            'pest.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

How many "scenarios" would you have to run to do this same analysis #thinkaboutit

In [ ]:
pst.set_res(os.path.join(m_d,"pest.{0}.sim.rei".format(pst.control_data.noptmax)))

In [ ]:
res = pst.res
res.loc[res.name.str.contains("gde"),:]

In [ ]:
pst.parrep(os.path.join(m_d,"pest.{0}.par".format(pst.control_data.noptmax)))

In [ ]:
par = pst.parameter_data
par.loc[wpar.parnme,:]

In [ ]:
wpar["i"] = wpar.idx1.astype(int)
wpar["j"] = wpar.idx2.astype(int)

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=m_d)
gwf = sim.get_model()
mg = gwf.modelgrid

In [ ]:
X,Y = mg.xcellcenters,mg.ycellcenters

In [ ]:
wpar["x"] = X[wpar.i,wpar.j]
wpar["y"] = Y[wpar.i,wpar.j]

In [ ]:
hkarr = np.zeros_like(X)
hkobs = obs.loc[obs.oname=="hk",:].copy()
hkobs["i"] = hkobs.i.astype(int)
hkobs["j"] = hkobs.j.astype(int)
hkarr[hkobs.i,hkobs.j] = np.log10(hkobs.obsval)
hkarr

In [ ]:
fig,ax = plt.subplots(1,1)
ax.pcolormesh(X,Y,hkarr)
ax.scatter(wpar.x,wpar.y,marker='.',s=80,c=par.loc[wpar.parnme,"parval1"].values,cmap="coolwarm")

## Reliability

Lets see how the posterior ensemble of realizations does in terms of feasibility

In [ ]:
oe_pt = pst.ies.get("obsen",pst.ies.phiactual.iteration.max())
oe_feas = oe_pt.loc[oe_pt[forecasts[1]]<80,:]
predev_gde_val = obs.loc[forecasts[0].replace("time:40151","time:1"),"obsval"]
# a gde flux value that plays to our advantage re uncertainty...
rhs_val = predev_gde_val - ((predev_gde_val * 0.2)*2)
oe_feas = oe_feas.loc[oe_feas[forecasts[0]]<rhs_val,:]
oe_feas = oe_feas.loc[oe_feas[dobs.obsnme[0]]<rhs_val,:]
oe_feas

So only a few realizations are feasible...we can calculate the "reliability" on the base scenario (with the standard rates)

In [ ]:
reliability = 100 * oe_feas.shape[0]/oe_pt.shape[0]
print(reliability," percent reliable")

yikes!  We probably dont want to go to press with these results...

In [ ]:
pst.pestpp_options["opt_par_stack"] = "pest.{0}.par.jcb".format(pst.ies.phiactual.iteration.max())
pst.pestpp_options["opt_risk"] = 0.65
pst.control_data.noptmax = 3
pst.write(os.path.join(t_d,"pest.pst"),version=2)

In [ ]:
pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-opt', #the PEST software version we want to run
                            'pest.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

In [ ]:
pst.parrep(os.path.join(m_d,"pest.{0}.par".format(pst.control_data.noptmax)))

In [ ]:
fig,ax = plt.subplots(1,1)
ax.pcolormesh(X,Y,hkarr)
ax.scatter(wpar.x,wpar.y,marker='.',s=80,c=par.loc[wpar.parnme,"parval1"].values,cmap="coolwarm")